### Limpieza y ajuste de datos iniciales

- Empezaremos el proceso realizando una ingeniería de datos a los datasets originales que fueron extraídos para realizar el proyecto.

- Los datasets empleados son:
 
 - steam_games.json: características de los juegos registrados en la plataforma STEAM
 - user_reviews.json: información de las reseñas efectuadas por usuarios en la plataforma
 - user_items.json: información de la relación de los usuarios con los juegos

- Los datasets resultantes serán guardados en la carpeta processed donde tendremos disponibles los datos, luego de su tratamiento y carga

- Para más información de los datos, consultar su diccionario de datos en la carpeta references>Diccionario de datos STEAM.xsl

In [1]:
# importamos las librerías
import json
import math
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import re
from textblob import TextBlob
import ast

# ver todas las columnas de los dataframes tratados
pd.set_option('display.max_columns', None)

#### Lectura y preprocesamiento del archivo steam_games.json

In [2]:
# Función boleana para validar si un valor es NaN
def is_nan(value):
    return math.isnan(value) if isinstance(value, float) else False

# Rutas de archivos
input_file_path = '../data/raw/steam_games.json'
output_file_path = '../data/interim/cleaned_steam_games.json'

# Lectura y limpieza del archivo JSON
with open(input_file_path, 'r', encoding='utf-8') as input_file, \
     open(output_file_path, 'w', encoding='utf-8') as output_file:

    for line in input_file:
        record = json.loads(line)
        if not all(is_nan(value) for value in record.values()):
            json.dump(record, output_file)
            output_file.write('\n')

print("Archivo steam_games limpiado y guardado.")

# Cargar el archivo JSON en un DataFrame
data = pd.read_json(output_file_path, lines=True)

# Manejar valores nulos en 'genres'
data['genres'] = data['genres'].apply(lambda x: x if isinstance(x, list) else [])

# Convertir la columna de fecha de lanzamiento
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')
data['release_year'] = data['release_date'].dt.year.fillna(0).astype(int)

# Explode 'genres' para tener una fila por género
df_games = data.explode('genres')

# Ahora, cada fila tendrá un único género, manteniendo los demás datos del juego
df_games.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del DataFrame resultante
print(df_games.head())

Archivo steam_games limpiado y guardado.
   publisher      genres             app_name                title  \
0  Kotoshiro      Action  Lost Summoner Kitty  Lost Summoner Kitty   
1  Kotoshiro      Casual  Lost Summoner Kitty  Lost Summoner Kitty   
2  Kotoshiro       Indie  Lost Summoner Kitty  Lost Summoner Kitty   
3  Kotoshiro  Simulation  Lost Summoner Kitty  Lost Summoner Kitty   
4  Kotoshiro    Strategy  Lost Summoner Kitty  Lost Summoner Kitty   

                                                 url release_date  \
0  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   
1  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   
2  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   
3  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   
4  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   

                                            tags  \
0  [Strategy, Action, Indie, Casual, Simulation]   
1  [Strategy, Actio

In [3]:
# Dejamos solo las columnas que usaremos para las consultas de los endpoints y para el modelo de ML
df_games = df_games.drop(['publisher', 'title', 'url', 'release_date', 'tags', 'reviews_url', 'specs', 'early_access'], axis=1)

# Mostrar como quedaría el dataframe
df_games.head()

,genres,app_name,price,id,developer,release_year
0,Action,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018


In [4]:
# Contar filas antes de la limpieza
filas_antes = df_games.shape[0]
print(f'Número de filas antes de la limpieza: {filas_antes}')

# Eliminar filas con valores NaN en las columnas 'genres', 'release_year' y 'developer'
df_games = df_games.dropna(subset=['genres', 'release_year', 'developer'])

# Contar filas después de la limpieza
filas_despues = df_games.shape[0]
print(f'Número de filas después de la limpieza: {filas_despues}')

# Calcular y mostrar el porcentaje de reducción
reduccion_porcentaje = ((filas_antes - filas_despues) / filas_antes) * 100
print(f'El dataset se redujo en un {reduccion_porcentaje:.2f}% después de la limpieza.')

Número de filas antes de la limpieza: 74837
Número de filas después de la limpieza: 71204
El dataset se redujo en un 4.85% después de la limpieza.


In [5]:
# Guardamos el dataframe procesado para los análisis posteriores
output_file_path = '../data/processed/processed_steam_games.csv'
df_games.to_csv(output_file_path, index=False)

output_file_path

'../data/processed/processed_steam_games.csv'

#### Lectura y preprocesamiento del archivo user_reviews.json

In [6]:
"""
Dada la complejidad del dataset, se crearon algunas funciones para hacer la lectura del archivo json.
En especial, el campo de las reseñas donde aparecen varios caracteres especiales y es necesario manejarlos
Cada función tiene su documentación correspondiente
"""

def convert_to_json_like(text):
    """
    Convierte el texto a un formato similar a un JSON válido.

    Esta función reemplaza las comillas simples por dobles y convierte los valores booleanos a minúsculas. 
    Estos cambios son necesarios para que el texto sea procesable como JSON, ya que JSON requiere comillas 
    dobles para las cadenas y los valores booleanos en minúsculas.

    Parámetros:
    - text (str): Texto que será convertido.

    Retorna:
    - str: Texto convertido en un formato similar a JSON.
    """
    text = text.replace("\'", "\"")
    text = text.replace(" True", " true")
    text = text.replace(" False", " false")
    return text

def escape_internal_quotes(json_like_text):
    """
    Escapa las comillas dobles internas en cadenas dentro de un texto similar a JSON.

    Esta función busca comillas dobles dentro de las cadenas de un texto que sigue una estructura similar a JSON 
    y las escapa. Esto es útil para prevenir errores de formato en el procesamiento de JSON, donde las comillas 
    dobles no escapadas pueden alterar la estructura esperada.

    Parámetros:
    - json_like_text (str): Texto en formato similar a JSON que será procesado.

    Retorna:
    - str: Texto con las comillas dobles internas escapadas.
    """
    escaped_text = re.sub(r'(?<!\\)"(?=[^"]*"[^"]*":)', '\\"', json_like_text)
    return escaped_text

def extract_data_line_by_line_trimmed(file_content):
    """
    Extrae y recorta información clave línea por línea del contenido del archivo.

    Esta función busca patrones específicos de datos (como ID de usuario, URL de usuario, ID del ítem, 
    fecha de publicación y reseña) en cada línea del contenido del archivo. Se asegura de eliminar 
    barras invertidas finales o caracteres no deseados en los datos extraídos. 

    Parámetros:
    - file_content (str): Contenido del archivo de donde se extraerán los datos.

    Retorna:
    - list: Lista de diccionarios con los datos extraídos y recortados.
    """
    user_id_pattern = re.compile(r'"user_id":\s*"([^"]+?)"(?=\s*,|\s*})')
    user_url_pattern = re.compile(r'"user_url":\s*"([^"]+?)"(?=\s*,|\s*})')
    item_id_pattern = re.compile(r'"item_id":\s*"([^"]+?)"(?=\s*,|\s*})')
    posted_pattern = re.compile(r'"posted":\s*"([^"]+?)"(?=\s*,|\s*})')
    review_pattern = re.compile(r'"review":\s*"([^"]+?)"(?=\s*,|\s*})')

    extracted_data = []
    current_data = {}

    for line in file_content.split('\n'):
        user_id_match = user_id_pattern.search(line)
        user_url_match = user_url_pattern.search(line)
        item_id_match = item_id_pattern.search(line)
        posted_match = posted_pattern.search(line)
        review_match = review_pattern.search(line)

        if user_id_match:
            current_data['user_id'] = user_id_match.group(1).rstrip('\\')
        if user_url_match:
            current_data['user_url'] = user_url_match.group(1).rstrip('\\')
        if item_id_match:
            current_data['item_id'] = item_id_match.group(1).rstrip('\\')
        if posted_match:
            current_data['posted'] = posted_match.group(1).rstrip('\\')
        if review_match:
            current_data['review'] = review_match.group(1).rstrip('\\')
            extracted_data.append(current_data.copy())
            current_data.clear()

    return extracted_data

# Lectura del archivo original
file_path = '../data/raw/user_reviews.json'
with open(file_path, 'r') as file:
    content = file.read()

# Uso de las funciones para carácteres especiales y de escape
json_like_text = convert_to_json_like(content)
escaped_text = escape_internal_quotes(json_like_text)

# Extracción de los datos y guardado
extracted_data = extract_data_line_by_line_trimmed(escaped_text)
df_reviews = pd.DataFrame(extracted_data)

# Muestra del dataframe
df_reviews.head()


,user_id,user_url,item_id,posted,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,"Posted November 5, 2011.",Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,251610,"Posted June 24, 2014.",Very fun little game to play when your bored o...
2,evcentric,http://steamcommunity.com/id/evcentric,248820,Posted February 3.,"Elegant integration of gameplay, story, world ..."
3,doctr,http://steamcommunity.com/id/doctr,250320,"Posted October 14, 2013.",This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,211420,"Posted April 15, 2014.",Git gud


#### Feature Engineering 

En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. 
Se creará la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. 
Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [7]:
def classify_sentiment(review_text):
    """
    Clasifica el sentimiento del texto de una reseña.

    Esta función analiza el sentimiento de un texto y lo clasifica en tres categorías:
    negativo (0), neutro (1) y positivo (2). Para ello, convierte primero el input a una cadena
    de texto, en caso de que no lo sea, y utiliza la polaridad de sentimiento obtenida a través
    de la biblioteca TextBlob.

    Parámetros:
    - review_text (str): Texto de la reseña que será analizado.

    Retorna:
    - int: Un valor entero representando la clasificación del sentimiento. Los posibles valores son
      0 (negativo), 1 (neutro) y 2 (positivo).

    Detalles adicionales:
    - Si el texto es 'nan' (o equivalente a una entrada nula/vacía), se clasifica como neutro.
    - La polaridad de sentimiento es un valor flotante entre -1 y 1, donde valores menores a -0.1
      indican un sentimiento negativo, mayores a 0.1 indican un sentimiento positivo, y cualquier
      valor entre -0.1 y 0.1 se considera neutro.
    """
    review_text = str(review_text)  # Convertir a cadena para manejar entradas no-cadena
    if review_text == 'nan':
        return 1  # Neutro para reseñas nulas o vacías

    # Análisis del sentimiento
    analysis = TextBlob(review_text)
    polarity = analysis.sentiment.polarity

    # Clasificación basada en la polaridad
    if polarity < -0.1:  # Sentimiento negativo
        return 0
    elif polarity > 0.1:  # Sentimiento positivo
        return 2
    else:
        return 1  # Sentimiento neutro

# Aplicar la clasificación de sentimiento al DataFrame
df_reviews['sentiment'] = df_reviews['review'].apply(classify_sentiment)

# Mostrar el DataFrame actualizado
df_reviews

,user_id,user_url,item_id,posted,review,sentiment
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,"Posted November 5, 2011.",Great atmosphere. The gunplay can be a bit chu...,1
1,js41637,http://steamcommunity.com/id/js41637,251610,"Posted June 24, 2014.",Very fun little game to play when your bored o...,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,Posted February 3.,"Elegant integration of gameplay, story, world ...",2
3,doctr,http://steamcommunity.com/id/doctr,250320,"Posted October 14, 2013.",This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,"Posted April 15, 2014.",Git gud,1
...,...,...,...,...,...,...
22339,JustMielThings,http://steamcommunity.com/id/JustMielThings,570,Posted May 20.,Good one,2
22340,Ghoustik,http://steamcommunity.com/id/Ghoustik,730,Posted June 17.,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...,1
22341,76561198310819422,http://steamcommunity.com/profiles/76561198310...,570,Posted June 23.,Well Done,1
22342,76561198312638244,http://steamcommunity.com/profiles/76561198312...,233270,Posted July 21.,this is a very fun and nice 80s themed shooter...,2


In [8]:
# Dejamos solo las columnas que usaremos para el análisis y los endpoints
df_reviews = df_reviews.drop(['user_url', 'posted', 'review'], axis=1)

# Mostrar como quedaría el dataframe
df_reviews.head()

,user_id,item_id,sentiment
0,76561197970982479,1250,1
1,js41637,251610,1
2,evcentric,248820,2
3,doctr,250320,2
4,maplemage,211420,1


In [9]:
# Guardamos el dataframe procesado para los análisis posteriores
output_file_path = '../data/processed/processed_users_reviews.csv'
df_reviews.to_csv(output_file_path, index=False)

output_file_path

'../data/processed/processed_users_reviews.csv'

#### Lectura y preprocesamiento del archivo user_items.json

In [10]:
def load_and_process_file(file_path):
    """
    Carga y procesa un archivo JSON línea por línea para crear un DataFrame.

    Esta función lee un archivo, donde cada línea es un registro JSON, y convierte cada línea en un diccionario.
    Posteriormente, agrega cada diccionario a una lista y utiliza esta lista para crear un DataFrame de pandas.

    Parámetros:
    - file_path (str): Ruta del archivo que se va a cargar y procesar.

    Retorna:
    - DataFrame: Un DataFrame de pandas con los datos del archivo.
    
    Nota:
    - Se asume que cada línea del archivo es una representación de diccionario válida.
    - Se utiliza 'ast.literal_eval' para convertir cada línea en un diccionario.
    """
    data_list = []
    with open(file_path, encoding='utf-8') as file:
        for line in file.readlines():
            data_list.append(ast.literal_eval(line))
    df = pd.DataFrame(data_list)
    return df

def explode_items_column(df):
    """
    Expande una columna de listas de un DataFrame, creando una fila para cada elemento de la lista.

    Esta función toma un DataFrame que tiene una columna ('items') cuyos valores son listas.
    Crea una nueva fila en el DataFrame para cada elemento de estas listas, replicando los valores 
    de las otras columnas en estas nuevas filas.

    Parámetros:
    - df (DataFrame): DataFrame original que contiene una columna con listas.

    Retorna:
    - DataFrame: Un nuevo DataFrame con la columna 'items' expandida.
    """
    exploded_df = df.explode('items')
    return exploded_df

# Cargar y procesar el archivo
file_path = '../data/raw/users_items.json'  # Reemplazar con la ruta real del archivo
df = load_and_process_file(file_path)

# Expandir la columna 'items'
exploded_df = explode_items_column(df)

# Convertir el diccionario en la columna 'items' en columnas separadas
df_users = pd.concat([exploded_df.drop(['items'], axis=1), exploded_df['items'].apply(pd.Series)], axis=1)

# Mostrar las primeras filas del DataFrame expandido
df_users

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0,NaN
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0,NaN
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0,NaN
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0,NaN


In [11]:
# Dejamos solo las columnas que usaremos para el análisis y los endpoints
df_users = df_users.drop(['steam_id', 'user_url', 'item_name', 'playtime_2weeks', 0], axis=1)

# Mostrar como quedaría el dataframe
df_users.head()

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
0,76561197970982479,277,20,0.0
0,76561197970982479,277,30,7.0
0,76561197970982479,277,40,0.0
0,76561197970982479,277,50,0.0


In [12]:
# Guardamos el dataframe procesado para los análisis posteriores
output_file_path = '../data/processed/processed_users_items.csv'
df_users.to_csv(output_file_path, index=False)

output_file_path

'../data/processed/processed_users_items.csv'

Ahora con los datasets procesados y limpios, organizaremos un dataset completo para responder a los endpoints.

2.0dataset-api.ipyn